# Data preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp drive/MyDrive/qa_data.jsonl.zip .

In [3]:
!unzip qa_data.jsonl.zip

Archive:  qa_data.jsonl.zip
  inflating: qa_data.jsonl           
  inflating: __MACOSX/._qa_data.jsonl  


In [4]:
!head -n 200000 qa_data.jsonl > train.jsonl
!head -n 250000 qa_data.jsonl | tail -n 50000 > valid.jsonl
!head -n 300000 qa_data.jsonl | tail -n 50000 > test.jsonl

In [7]:
!pip install youtokentome

     |████████████████████████████████| 1.7 MB 4.2 MB/s 


In [8]:
!head -n 500000 qa_data.jsonl | sed 's/[^0-9а-яА-Я \-\.\?]//g' | sed 's/  / /g' > forbpe.txt

# Imports

In [23]:
import pandas as pd
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [42]:
from torchtext.legacy.data import BucketIterator

In [10]:
import youtokentome as yttm

# Подготовка

In [11]:
%%time
vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'
yttm.BPE.train(data='forbpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 17.8 s, sys: 2.84 s, total: 20.6 s
Wall time: 12.5 s


In [12]:
tokenizer = yttm.BPE(model=model_path)

In [19]:
(PAD_TOKEN,
UNK_TOKEN,
START_TOKEN,
END_TOKEN) = tokenizer.vocab()[:4]

In [41]:
tokenizer.subword_to_id(PAD_TOKEN)

0

In [42]:
class QADataset(Dataset):
    def __init__(self, filename, _tokenizer=None, MAX_LEN=64):
        super().__init__()
        if _tokenizer is None:
            _tokenizer = tokenizer
        self._tokenizer = _tokenizer
        questions = []
        responses = []
        self.length: int
        with open(filename, "r", encoding="utf-8") as file:
            for line in file:
                line = line.strip("\n").strip("\r")
                line_dict = json.loads(line)
                question = line_dict["question"]
                response = line_dict["responses"]
                if len(response) == 0:
                    continue
                questions.append(
                    self._tokenize(question, MAX_LEN)
                )
                responses.append(
                    self._tokenize(response[0], MAX_LEN)
                )
            else:
                assert len(questions) == len(responses)
                self.length = len(questions)
        self.questions = torch.nn.utils.rnn.pad_sequence(
            questions,
            batch_first=True,
            padding_value=self._tokenizer.subword_to_id(PAD_TOKEN)
        )
        self.responses = torch.nn.utils.rnn.pad_sequence(
            questions,
            batch_first=True,
            padding_value=self._tokenizer.subword_to_id(PAD_TOKEN)
        )
        
    def __len__(self):
        return self.length

    def __getitem__(self, item):
        return self.questions[item], self.responses[item]

    def _tokenize(self, text, max_len):
        return torch.LongTensor(
            self._tokenizer.encode(text, bos=True, eos=True)[:max_len]
        )